In [ ]:
import pandas as pd
import time
from dateutil.parser import parse as dateutil_parser
from pprint import pprint
#import streamlit as st

import datetime
#import yfinance as yf

# SDK Packages
import fds.sdk.Formula # To install, please use "pip install fds.sdk.Formula==2.2.1"
from fds.sdk.Formula.apis import * 
from fds.sdk.Formula.models import *
from fds.sdk.utils.authentication import ConfidentialClient # To install, please use "pip install fds.sdk.utils"

In [ ]:
import sys
sys.path.append('../')
from fds_credentials import username,password

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import os

def fetch_market_data():
    # Make sure data is not pulled in scientific notation
    pd.options.display.float_format = '{:,.2f}'.format

    # username = st.secrets.FDS_API.username
    # password = st.secrets.FDS_API.password

    # Basic authentication: FactSetApiKey 
    configuration = fds.sdk.Formula.Configuration(
        username=username, password=password
    )

    with fds.sdk.Formula.ApiClient(configuration) as api_client:
        # Create Instance
        api_instance = TimeSeriesApi(api_client)

        # Define the filename
        filename = "SPX_data.csv"

        # Check if the file already exists
        if os.path.exists(filename):
            # Load existing data
            existing_data = pd.read_csv(filename)
            # Get the last date available in the existing data
            last_date = existing_data['date'].max()
            # Convert last date to datetime object
            last_date = datetime.strptime(last_date, '%Y-%m-%d')
            # Set start_date to the day after the last_date
            start_date = last_date + timedelta(days=1)
            # Format dates for the API call
            start_date_str = start_date.strftime('%m/%d/%Y')
        else:
            # If the file doesn't exist, set start date to a year ago
            start_date = datetime.now() - timedelta(days=1460)
            start_date_str = start_date.strftime('%m/%d/%Y')

        # Set end date to today
        end_date_str = datetime.now().strftime('%m/%d/%Y')

        # Request Object to Define Parameters
        time_series_request = TimeSeriesRequest(
            data=TimeSeriesRequestData(
                ids = ["SPMO"],
                formulas = [
                    'FSYM_TICKER_EXCHANGE(0,"ID")',
                    f"P_PRICE({start_date_str},{end_date_str})"
                    ],
            fsymId = "Y",
            flatten = "Y"
        ),
    )

        # Send Request
        api_response_wrapper = api_instance.get_time_series_data_for_list(time_series_request)
        api_response = api_response_wrapper.get_response_200()

        # Convert to Pandas DataFrame
        new_data = pd.DataFrame(api_response.to_dict()['data'])

        # Drop and rename columns
        new_data = new_data.rename(columns={
            'FSYM_TICKER_EXCHANGE(0,"ID")': "Ticker",
            f"P_PRICE({start_date_str},{end_date_str})": "Price"
        })

        # Reorder the columns
        new_data = new_data[['date', 'Ticker', 'Price']]

        # Check if the file already exists
        if os.path.exists(filename):
            # Append new data
            combined_data = pd.concat([existing_data, new_data], ignore_index=True)
        else:
            # If the file doesn't exist, use the new data as the combined data
            combined_data = new_data

        # Convert 'date' column to datetime format
        combined_data['date'] = pd.to_datetime(combined_data['date'])

        # Sort the combined data by Ticker (alphabetically) and then by date (ascending)
        combined_data = combined_data.sort_values(by=['Ticker', 'date'])

        # Drop duplicates, keeping the last entry (most recent) for each Ticker on each date
        combined_data = combined_data.drop_duplicates(subset=['date', 'Ticker'], keep='last')

        # Save the sorted and cleaned combined data
        combined_data.to_csv(filename, index=False)

    return new_data

fetch_market_data()